# TODO
- [ ] Add illustrations for the concept (dinov2, instance retrieval, faiss)
- [ ] There are a lot of possible optimizations I can apply for both search and encoding, but I don't have time for now
- [ ] Add a pipeline to give a nice caption according to the input image and the nearest image with NLP
- [ ] Upload it into HuggingFace space as a demo
- [ ] Save Faiss state

# References & useful resources


In [ ]:
# !git clone https://github.com/facebookresearch/dinov2.git

In [1]:
# %cd /dinov2
# !pip install -r requirements.txt
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.2 MB/s eta 0:00:00


In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.9 MB/s eta 0:

In [3]:
# !pip install faiss-gpu
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
import faiss
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import ImageFolder
import warnings

warnings.filterwarnings("ignore")
import gradio as gr

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the DINO model
dino = torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14")
dino.to(device)
dino.eval()

# Define the image transformations
image_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),  # Convert to RGB format
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


# Function to extract features from an image
def extract_features(image):
    image_tensor = image_transforms(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = dino(image_tensor).float()
    return features


# Function to perform nearest neighbor search
def nearest_neighbor_search(query_features, dataset_features, k=10):
    index = faiss.IndexFlatL2(query_features.shape[1])
    dataset_features = torch.cat(dataset_features, dim=0).cpu().numpy()
    index.add(dataset_features)
    distances, indices = index.search(query_features.cpu().numpy(), k)
    return indices


# Function to return the nearest neighbor images
def return_nearest_neighbor_images(query_image):
    query_features = extract_features(query_image)
    indices = nearest_neighbor_search(query_features, dataset_features)
    nearest_neighbor_image_paths = []
    for i in range(k):
        image_path = os.path.join(dataset_path, dataset_images[indices[0][i]])
        nearest_neighbor_image_paths.append(image_path)
    return nearest_neighbor_image_paths


# Gradio interface
dataset_path = "/content/drive/MyDrive/newpaper/Hader/dataset/jeans/"
dataset_images = os.listdir(dataset_path)
dataset_features = []
for filename in dataset_images:
    image_path = os.path.join(dataset_path, filename)
    if (
        filename.endswith(".png")
        or filename.endswith(".jpg")
        or filename.endswith(".jpeg")
    ):
        image = Image.open(image_path)
        features = extract_features(image)
        dataset_features.append(features)

k = 5  # Number of nearest neighbors to retrieve

inputs = gr.inputs.Image(type="pil", label="Input Image")
# outputs = gr.outputs.Image(type="pil", label="Returned Images", multiple=True)

outputs = gr.Gallery()

demo = gr.Interface(
    fn=return_nearest_neighbor_images,
    inputs=inputs,
    outputs=outputs,
    title="Recommendation Fusion",
    description="Please, uploading an image to  get the recommendation images.",
)

demo.launch()

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>